In [1]:
import geemap, ee

In [2]:
Map = geemap.Map()
Map.add_basemap("SATELLITE")
# Map

## Change to Sentinel 2 TOA and check cloud covers; change ROI and clustering ROI for better accuracy!

In [3]:
# point = ee.Geometry.Point([-122.4439, 37.7538])
# point = ee.Geometry.Point([-87.7719, 41.8799]) # Michigan
# point = ee.Geometry.Point([18.08, 59.32]) # Stockholm
# region = ee.Geometry.BBox(17.82, 59.22, 18.34, 59.42) # Stockholm
region = ee.Geometry.BBox(17.70, 59.20, 18.70, 59.60) # Stockholm

image = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(region)
    .filterDate('2023-06-01', '2023-08-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)
    .mosaic()
    .unitScale(0, 10000)
    .clip(region) # Don't clip as we want to analyse the whole image
    # .select('B[1-7]')
)

# True Color Display:
vis_params = {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}

Map.centerObject(region, 10)
Map.addLayer(image, vis_params, "Sentinel-2")

props = geemap.image_props(image)
props.getInfo()
# props.get('IMAGE_DATE').getInfo() # For LandSat-8; Sentinel-2 does not have these image properties
# props.get('CLOUD_COVER').getInfo() # For LandSat-8; Sentinel-2 does not have these image properties

# Construct Training Data Set:

In [4]:
ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
ndwi = image.normalizedDifference(['B3', 'B8']).rename('ndwi')
input = image.addBands([ndvi, ndwi])

# Define the input features for the classifier
# inputBands = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'ndvi', 'ndwi']
inputBands = ['B2', 'B3', 'B4', 'B8', 'B11', 'ndvi', 'ndwi']

In [5]:
# Make the training dataset.
training = input.select(inputBands).sample(
    **{
        'region': region,
        'scale': 30,
        'numPixels': 5000, # Default, given
        # 'numPixels': 10000,
        # 'seed': 0,
        # 'geometries': True,  # Set this to False to ignore geometries
    }
)

# Plot the training sample data (not really necessary)
# Map.addLayer(training, {}, 'training', False)
# Map

In [6]:
# Instantiate the clusterer and train it.
n_clusters = 15
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [7]:
# Cluster the input using the trained clusterer.
result_unclassified = input.select(inputBands).cluster(clusterer)
# geemap.image_props(result)

# # Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'clusters')
# Map.addLayer(result, {}, 'clusters')
# Map

# Unlabelled cluster:

In [8]:
# Define 15 colors to distinguish the clusters and look at what they are!
# legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
# legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
legend_keys = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14']
legend_colors = ['#e6194B','#3cb44b','#ffe119','#4363d8','#f58231','#911eb4','#42d4f4','#f032e6','#bfef45','#fabed4','#469990','#9A6324','#800000','#808000','#000075']

Map.addLayer(
    result_unclassified, {'min': 0, 'max': 14, 'palette': legend_colors}, 'Unclassified clusters'
)
Map.add_legend(
    keys=legend_keys, colors=legend_colors, position='bottomright'
)
Map

Map(center=[59.399925146053874, 18.19999999999986], controls=(WidgetControl(options=['position', 'transparent_…

# Cluster-Class Remapping:
## 0 - Urban: 7 (darker/road), 9 (metallic rooftop or cloud)
## 1 - Water: 2 (deep), 13 (shallow), 14 (?, close to outer seas)
## 2 - Vegetation: 0 (larger), 1 (wet?), 3, 4 (slight veg) , 5 (tall), 8 (very wet), 11 (more disperse veg)
## 3 - Bare Field: 6 (a bit more veg), 10 (barer), 12 (even more barer)

In [9]:
# 0 - Urban
# 1 - Water
# 2 - Forest/Veg
# 3 - Bare Fields

# Define Classification Correspondance Scheme: ['Urban', 'Water', 'Vegetation',  'Bare Field'] = [0,1,2,3]
legend_keys_classified = ['Urban', 'Water', 'Vegetation', 'Bare Field']
legend_colors_classified = ['#a9a9a9','#4363d8', '#3cb44b', '#ffe119']

# Remap the spectral classes from clusters:
fromClusters = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
toClasses = [2, 2, 1, 2, 2, 2, 3, 0, 2, 0, 3, 2, 3, 1, 1]

# Reclassify the map
result_classified = result_unclassified.remap(fromClusters, toClasses)

Map.addLayer(
    result_classified, {'min': 0, 'max': 3, 'palette': legend_colors_classified}, 'Labelled clusters'
)
Map.add_legend(
    keys=legend_keys_classified, colors=legend_colors_classified, position='bottomleft'
)

# Image Export (optional)
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

geemap.ee_export_image(result, filename=out_file, scale=90)
geemap.ee_export_image_to_drive(
    result, description='clusters', folder='export', scale=90
)

# Split bar (problematic, to remove)
# import geemap.foliumap as gf
import geemap as gf
Map2 = gf.Map()
Map2.centerObject(region, 8)
Map2.add_basemap("SATELLITE")

tile_img = gf.ee_tile_layer(image, vis_params, 'Sentinel-2 True Colour')
tile_cluster = gf.ee_tile_layer(result, {'min': 0, 'max': 14, 'palette': legend_colors}, 'Sentinel-2 Clustered Unclassified')


Map2.split_map(left_layer=tile_img, right_layer=tile_cluster)
Map2.addLayerControl

Map2.add_legend(
    labels=legend_keys, colors=legend_colors, position='bottomright'
)
# Map2.add_legend(
#     keys=legend_keys, colors=legend_colors, position='bottomright'
# )

Map2

# Unsupervised Classification - Accuracy Assessment:

# Import User Labelled Polygons from Supervised Classification for Accuracy Assessment:
# Class 0 - Water:
water = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.1287605285643, 59.420668377041466],
                  [18.1287605285643, 59.41916191061126],
                  [18.132365417480315, 59.41916191061126],
                  [18.132365417480315, 59.420668377041466]]], None, False),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.155077293266338, 59.390922117625365],
                  [18.155077293266338, 59.3900917489387],
                  [18.15679390703587, 59.3900917489387],
                  [18.15679390703587, 59.390922117625365]]], None, False),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.103407218803447, 59.36862621806919],
                  [18.103407218803447, 59.36823262938389],
                  [18.104523017753642, 59.36823262938389],
                  [18.104523017753642, 59.36862621806919]]], None, False),
            {
              "landcover": 0,
              "system:index": "2"
            })])
# Class 1 - Vegetation:
vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01369561111225, 59.34494428938679],
                  [18.01369561111225, 59.344594188824566],
                  [18.01425351058735, 59.344594188824566],
                  [18.01425351058735, 59.34494428938679]]], None, False),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.014038933866157, 59.341924553350076],
                  [18.014038933866157, 59.341661954918315],
                  [18.0149830714394, 59.341661954918315],
                  [18.0149830714394, 59.341924553350076]]], None, False),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.02884909415171, 59.36172374248307],
                  [18.02884909415171, 59.36128633246986],
                  [18.030436961888526, 59.36128633246986],
                  [18.030436961888526, 59.36172374248307]]], None, False),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.060434787511085, 59.3606520779365],
                  [18.060434787511085, 59.35982096784676],
                  [18.061378925084327, 59.35982096784676],
                  [18.061378925084327, 59.3606520779365]]], None, False),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.064468829869483, 59.363320241188205],
                  [18.064468829869483, 59.362707894399826],
                  [18.065370052098487, 59.362707894399826],
                  [18.065370052098487, 59.363320241188205]]], None, False),
            {
              "landcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.05060717368052, 59.36286098213282],
                  [18.05060717368052, 59.36255480597621],
                  [18.051122157811378, 59.36255480597621],
                  [18.051122157811378, 59.36286098213282]]], None, False),
            {
              "landcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.071378200291846, 59.35533699654806],
                  [18.071378200291846, 59.3545713812703],
                  [18.07257982993052, 59.3545713812703],
                  [18.07257982993052, 59.35533699654806]]], None, False),
            {
              "landcover": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.07506891989634, 59.355555740598646],
                  [18.07506891989634, 59.35481200507544],
                  [18.075712650059913, 59.35481200507544],
                  [18.075712650059913, 59.355555740598646]]], None, False),
            {
              "landcover": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.080274348227597, 59.364458675875035],
                  [18.080274348227597, 59.36371513542555],
                  [18.081046824423886, 59.36371513542555],
                  [18.081046824423886, 59.364458675875035]]], None, False),
            {
              "landcover": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.083793406455136, 59.36660173071935],
                  [18.083793406455136, 59.36585823722938],
                  [18.08473754402838, 59.36585823722938],
                  [18.08473754402838, 59.36660173071935]]], None, False),
            {
              "landcover": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.091518168418027, 59.36375887355005],
                  [18.091518168418027, 59.36314653467683],
                  [18.09194732186041, 59.36314653467683],
                  [18.09194732186041, 59.36375887355005]]], None, False),
            {
              "landcover": 1,
              "system:index": "10"
            })])
# Class 2 - Urban:
urban = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.053864163954678, 59.332220234026025],
                  [18.053864163954678, 59.33169488507395],
                  [18.05482975920004, 59.33169488507395],
                  [18.05482975920004, 59.332220234026025]]], None, False),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.126311076754277, 59.3412618738503],
                  [18.126311076754277, 59.34058347717914],
                  [18.127383960360234, 59.34058347717914],
                  [18.127383960360234, 59.3412618738503]]], None, False),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.08801953187299, 59.30215086123935],
                  [18.08801953187299, 59.30188795534564],
                  [18.088427227643255, 59.30188795534564],
                  [18.088427227643255, 59.30215086123935]]], None, False),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.091216725018743, 59.304976971271074],
                  [18.091216725018743, 59.30459359801732],
                  [18.091667336133245, 59.30459359801732],
                  [18.091667336133245, 59.304976971271074]]], None, False),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.098877113965276, 59.30182222855471],
                  [18.098877113965276, 59.30118686302864],
                  [18.09977833619428, 59.30118686302864],
                  [18.09977833619428, 59.30182222855471]]], None, False),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.110163849499944, 59.308306660060424],
                  [18.110163849499944, 59.307978086848095],
                  [18.110936325696233, 59.307978086848095],
                  [18.110936325696233, 59.308306660060424]]], None, False),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.066390198376897, 59.307101876095324],
                  [18.066390198376897, 59.30686091418073],
                  [18.06681935181928, 59.30686091418073],
                  [18.06681935181928, 59.307101876095324]]], None, False),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.054288071301702, 59.30118686302864],
                  [18.054288071301702, 59.301011587691676],
                  [18.05463139405561, 59.301011587691676],
                  [18.05463139405561, 59.30118686302864]]], None, False),
            {
              "landcover": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.075874489453557, 59.306357079206215],
                  [18.075874489453557, 59.30611611201477],
                  [18.076260727551702, 59.30611611201477],
                  [18.076260727551702, 59.306357079206215]]], None, False),
            {
              "landcover": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.08102433076215, 59.29939024801239],
                  [18.08102433076215, 59.29859050471372],
                  [18.081496399548772, 59.29859050471372],
                  [18.081496399548772, 59.29939024801239]]], None, False),
            {
              "landcover": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01549805557026, 59.34995470818497],
                  [18.01549805557026, 59.349123336140345],
                  [18.01674260055317, 59.349123336140345],
                  [18.01674260055317, 59.34995470818497]]], None, False),
            {
              "landcover": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.029145135038032, 59.34636653580566],
                  [18.029145135038032, 59.3458632861902],
                  [18.030132187955513, 59.3458632861902],
                  [18.030132187955513, 59.34636653580566]]], None, False),
            {
              "landcover": 2,
              "system:index": "11"
            })])
# Class 3 - Bare Field
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[17.983144676571825, 59.48949503893704],
                  [17.983144676571825, 59.48892854715184],
                  [17.983960068112353, 59.48892854715184],
                  [17.983960068112353, 59.48949503893704]]], None, False),
            {
              "landcover": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.999624168759325, 59.48749048685693],
                  [17.999624168759325, 59.48659711552939],
                  [18.00129786718462, 59.48659711552939],
                  [18.00129786718462, 59.48749048685693]]], None, False),
            {
              "landcover": 3,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.002842819577197, 59.49544262887063],
                  [18.002842819577197, 59.494898021360086],
                  [18.003701126461962, 59.494898021360086],
                  [18.003701126461962, 59.49544262887063]]], None, False),
            {
              "landcover": 3,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.04191724050615, 59.494505898511925],
                  [18.04191724050615, 59.49410287861594],
                  [18.042582428341845, 59.49410287861594],
                  [18.042582428341845, 59.494505898511925]]], None, False),
            {
              "landcover": 3,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.88718596685503, 59.49958135881831],
                  [17.88718596685503, 59.49840513982099],
                  [17.889331734066943, 59.49840513982099],
                  [17.889331734066943, 59.49958135881831]]], None, False),
            {
              "landcover": 3,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.887774834533488, 59.591804275916374],
                  [17.887774834533488, 59.59067471774544],
                  [17.88949144830302, 59.59067471774544],
                  [17.88949144830302, 59.591804275916374]]], None, False),
            {
              "landcover": 3,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.90395391931132, 59.58652543804897],
                  [17.90395391931132, 59.58411384133924],
                  [17.905756363769328, 59.58411384133924],
                  [17.905756363769328, 59.58652543804897]]], None, False),
            {
              "landcover": 3,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.104920177676632, 59.342211009243925],
                  [18.104920177676632, 59.341871822081195],
                  [18.105413704135373, 59.341871822081195],
                  [18.105413704135373, 59.342211009243925]]], None, False),
            {
              "landcover": 3,
              "system:index": "7"
            })])

Map.addLayer(water, {}, "water")
Map.addLayer(vegetation, {}, "vegetation")
Map.addLayer(urban, {}, "urban")
Map.addLayer(bare, {}, "bare")
Map

# New imports - more training regions:
# Class 0 - Water:
water = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.1287605285643, 59.420668377041466],
                  [18.1287605285643, 59.41855057178141],
                  [18.132880401611175, 59.41855057178141],
                  [18.132880401611175, 59.420668377041466]]], None, False),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.155077293266338, 59.390922117625365],
                  [18.155077293266338, 59.388518363011805],
                  [18.160570457328838, 59.388518363011805],
                  [18.160570457328838, 59.390922117625365]]], None, False),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.101690605033916, 59.36862621806919],
                  [18.101690605033916, 59.36512749188252],
                  [18.107784583915752, 59.36512749188252],
                  [18.107784583915752, 59.36862621806919]]], None, False),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.078485317078293, 59.322705646967805],
                  [18.078485317078293, 59.32136999522701],
                  [18.0823047827155, 59.32136999522701],
                  [18.0823047827155, 59.322705646967805]]], None, False),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.056233263540545, 59.32401824428743],
                  [18.056233263540545, 59.322047668267054],
                  [18.059752321768084, 59.322047668267054],
                  [18.059752321768084, 59.32401824428743]]], None, False),
            {
              "landcover": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.990464791118104, 59.330316839397135],
                  [17.990464791118104, 59.32834662857648],
                  [17.994155510722596, 59.32834662857648],
                  [17.994155510722596, 59.330316839397135]]], None, False),
            {
              "landcover": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.98780403977533, 59.34467403662359],
                  [17.98780403977533, 59.34252959696393],
                  [17.991494759379822, 59.34252959696393],
                  [17.991494759379822, 59.34467403662359]]], None, False),
            {
              "landcover": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.25405792757162, 59.37983915870227],
                  [18.25405792757162, 59.37826529692743],
                  [18.25731949373373, 59.37826529692743],
                  [18.25731949373373, 59.37983915870227]]], None, False),
            {
              "landcover": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.789414237013364, 59.3573267225729],
                  [17.789414237013364, 59.35277679343631],
                  [17.797653983107114, 59.35277679343631],
                  [17.797653983107114, 59.3573267225729]]], None, False),
            {
              "landcover": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.732422659864927, 59.4209617222526],
                  [17.732422659864927, 59.41711904193939],
                  [17.736542532911802, 59.41711904193939],
                  [17.736542532911802, 59.4209617222526]]], None, False),
            {
              "landcover": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.76881487177899, 59.44645209758956],
                  [17.76881487177899, 59.4440086473052],
                  [17.779114554396177, 59.4440086473052],
                  [17.779114554396177, 59.44645209758956]]], None, False),
            {
              "landcover": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.316071341505552, 59.30794491711008],
                  [18.316071341505552, 59.30373889551123],
                  [18.324311087599302, 59.30373889551123],
                  [18.324311087599302, 59.30794491711008]]], None, False),
            {
              "landcover": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.161232779493833, 59.32695314970951],
                  [18.161232779493833, 59.32485137940307],
                  [18.166382620802427, 59.32485137940307],
                  [18.166382620802427, 59.32695314970951]]], None, False),
            {
              "landcover": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.280552863119695, 59.38369091578032],
                  [18.280552863119695, 59.37980028875377],
                  [18.289565085409734, 59.37980028875377],
                  [18.289565085409734, 59.38369091578032]]], None, False),
            {
              "landcover": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.033239451895742, 59.50996377764133],
                  [18.033239451895742, 59.50482454456448],
                  [18.0418225207434, 59.50482454456448],
                  [18.0418225207434, 59.50996377764133]]], None, False),
            {
              "landcover": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.09538833818113, 59.31835313775008],
                  [18.09538833818113, 59.31728010393107],
                  [18.098478242966287, 59.31728010393107],
                  [18.098478242966287, 59.31835313775008]]], None, False),
            {
              "landcover": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.09163324556028, 59.32151729537798],
                  [18.09163324556028, 59.31990788176361],
                  [18.09493772706663, 59.31990788176361],
                  [18.09493772706663, 59.32151729537798]]], None, False),
            {
              "landcover": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.04970794169684, 59.30631471939471],
                  [18.04970794169684, 59.30451837520289],
                  [18.05350594966193, 59.30451837520289],
                  [18.05350594966193, 59.30631471939471]]], None, False),
            {
              "landcover": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.033850722000796, 59.30993997876862],
                  [18.033850722000796, 59.30795763406247],
                  [18.037112288162906, 59.30795763406247],
                  [18.037112288162906, 59.30993997876862]]], None, False),
            {
              "landcover": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01400237529059, 59.312108378207],
                  [18.01400237529059, 59.31138559375791],
                  [18.01655583827277, 59.31138559375791],
                  [18.01655583827277, 59.312108378207]]], None, False),
            {
              "landcover": 0,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.00598936987815, 59.399749007250186],
                  [18.00598936987815, 59.39735680230275],
                  [18.009744462499, 59.39735680230275],
                  [18.009744462499, 59.399749007250186]]], None, False),
            {
              "landcover": 0,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.991934594640114, 59.415146831337566],
                  [17.991934594640114, 59.41191496074184],
                  [17.99704152060447, 59.41191496074184],
                  [17.99704152060447, 59.415146831337566]]], None, False),
            {
              "landcover": 0,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.05465537024436, 59.40777640587995],
                  [18.05465537024436, 59.40706111625854],
                  [18.05623250914512, 59.40706111625854],
                  [18.05623250914512, 59.40777640587995]]], None, False),
            {
              "landcover": 0,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.060865223528232, 59.37764527486218],
                  [18.060865223528232, 59.374934514359545],
                  [18.067259609819736, 59.374934514359545],
                  [18.067259609819736, 59.37764527486218]]], None, False),
            {
              "landcover": 0,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.022541821123447, 59.37646480901623],
                  [18.022541821123447, 59.37438796382078],
                  [18.025545895220127, 59.37438796382078],
                  [18.025545895220127, 59.37646480901623]]], None, False),
            {
              "landcover": 0,
              "system:index": "24"
            })])
# Class 1 - Vegetation:
vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01369561111225, 59.34494428938679],
                  [18.01369561111225, 59.344594188824566],
                  [18.01425351058735, 59.344594188824566],
                  [18.01425351058735, 59.34494428938679]]], None, False),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.014038933866157, 59.341924553350076],
                  [18.014038933866157, 59.341661954918315],
                  [18.0149830714394, 59.341661954918315],
                  [18.0149830714394, 59.341924553350076]]], None, False),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.02884909415171, 59.36211740667467],
                  [18.02884909415171, 59.36028026804388],
                  [18.032239406346534, 59.36028026804388],
                  [18.032239406346534, 59.36211740667467]]], None, False),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.060434787511085, 59.36064661017352],
                  [18.060434787511085, 59.35982096784676],
                  [18.061969011067603, 59.35982096784676],
                  [18.061969011067603, 59.36064661017352]]], None, False),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.063610522984717, 59.36341865303429],
                  [18.063610522984717, 59.362489196440386],
                  [18.066153257130836, 59.362489196440386],
                  [18.066153257130836, 59.36341865303429]]], None, False),
            {
              "landcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.049169509648536, 59.36349519538389],
                  [18.049169509648536, 59.36255480597621],
                  [18.051122157811378, 59.36255480597621],
                  [18.051122157811378, 59.36349519538389]]], None, False),
            {
              "landcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.071378200291846, 59.35682442829472],
                  [18.071378200291846, 59.3545713812703],
                  [18.07674261832163, 59.3545713812703],
                  [18.07674261832163, 59.35682442829472]]], None, False),
            {
              "landcover": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.039706676243995, 59.35710878282482],
                  [18.039706676243995, 59.355818231598626],
                  [18.04232451224253, 59.355818231598626],
                  [18.04232451224253, 59.35710878282482]]], None, False),
            {
              "landcover": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.078321700064755, 59.364546150150915],
                  [18.078321700064755, 59.363693266342196],
                  [18.081046824423886, 59.363693266342196],
                  [18.081046824423886, 59.364546150150915]]], None, False),
            {
              "landcover": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.082505946127988, 59.36660173071935],
                  [18.082505946127988, 59.36565049348929],
                  [18.08473754402838, 59.36565049348929],
                  [18.08473754402838, 59.36660173071935]]], None, False),
            {
              "landcover": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.090574030844785, 59.36484137416718],
                  [18.090574030844785, 59.36314653467683],
                  [18.093513731925107, 59.36314653467683],
                  [18.093513731925107, 59.36484137416718]]], None, False),
            {
              "landcover": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.121429990560365, 59.328055748769714],
                  [18.121429990560365, 59.32693923570562],
                  [18.123361181051088, 59.32693923570562],
                  [18.123361181051088, 59.328055748769714]]], None, False),
            {
              "landcover": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.13576371553595, 59.32540670706443],
                  [18.13576371553595, 59.32398358287265],
                  [18.13825280550177, 59.32398358287265],
                  [18.13825280550177, 59.32540670706443]]], None, False),
            {
              "landcover": 1,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.048286438822153, 59.31351383988241],
                  [18.048286438822153, 59.31245160437228],
                  [18.050754071115854, 59.31245160437228],
                  [18.050754071115854, 59.31351383988241]]], None, False),
            {
              "landcover": 1,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.12301317314356, 59.338600584300885],
                  [18.12301317314356, 59.33581009907216],
                  [18.128549252550297, 59.33581009907216],
                  [18.128549252550297, 59.338600584300885]]], None, False),
            {
              "landcover": 1,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.909927846933623, 59.40869750046596],
                  [17.909927846933623, 59.406819201511915],
                  [17.913446905161162, 59.406819201511915],
                  [17.913446905161162, 59.40869750046596]]], None, False),
            {
              "landcover": 1,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.912331106210967, 59.399588962709345],
                  [17.912331106210967, 59.39823448657892],
                  [17.915592672373077, 59.39823448657892],
                  [17.915592672373077, 59.399588962709345]]], None, False),
            {
              "landcover": 1,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.88470670100653, 59.40917671266396],
                  [17.88470670100653, 59.4071892351965],
                  [17.88865491267645, 59.4071892351965],
                  [17.88865491267645, 59.40917671266396]]], None, False),
            {
              "landcover": 1,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.979135726798837, 59.411884660487154],
                  [17.979135726798837, 59.410661709009055],
                  [17.98059484850294, 59.410661709009055],
                  [17.98059484850294, 59.411884660487154]]], None, False),
            {
              "landcover": 1,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.985658859123056, 59.409657108689025],
                  [17.985658859123056, 59.40822659376949],
                  [17.988405441154306, 59.40822659376949],
                  [17.988405441154306, 59.409657108689025]]], None, False),
            {
              "landcover": 1,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.014604199818365, 59.407413737428996],
                  [18.014604199818365, 59.40660015384685],
                  [18.016084779194586, 59.40660015384685],
                  [18.016084779194586, 59.407413737428996]]], None, False),
            {
              "landcover": 1,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.010623801640264, 59.405022073609665],
                  [18.010623801640264, 59.40459068248083],
                  [18.011525023869268, 59.40459068248083],
                  [18.011525023869268, 59.405022073609665]]], None, False),
            {
              "landcover": 1,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.99300747824607, 59.40285097587157],
                  [17.99300747824607, 59.401540319550485],
                  [17.996075925359108, 59.401540319550485],
                  [17.996075925359108, 59.40285097587157]]], None, False),
            {
              "landcover": 1,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.995432195195534, 59.39848192429468],
                  [17.995432195195534, 59.397040013642794],
                  [17.997642335423805, 59.397040013642794],
                  [17.997642335423805, 59.39848192429468]]], None, False),
            {
              "landcover": 1,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.988222417363502, 59.39558711737275],
                  [17.988222417363502, 59.39446189926777],
                  [17.990196523198463, 59.39446189926777],
                  [17.990196523198463, 59.39558711737275]]], None, False),
            {
              "landcover": 1,
              "system:index": "24"
            })])
# Class 2 - Urban:
urban = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.052726907332364, 59.33283313087029],
                  [18.052726907332364, 59.33097251700143],
                  [18.056632203658047, 59.33097251700143],
                  [18.056632203658047, 59.33283313087029]]], None, False),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.123178256624882, 59.34183084125545],
                  [18.123178256624882, 59.33977376018854],
                  [18.1297013889491, 59.33977376018854],
                  [18.1297013889491, 59.34183084125545]]], None, False),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.08752600541425, 59.302238496085714],
                  [18.08752600541425, 59.30166886554873],
                  [18.088727635052923, 59.30166886554873],
                  [18.088727635052923, 59.302238496085714]]], None, False),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.091452759412054, 59.30506459883667],
                  [18.091452759412054, 59.304681226570594],
                  [18.091903370526556, 59.304681226570594],
                  [18.091903370526556, 59.30506459883667]]], None, False),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.09853379121137, 59.30164695649144],
                  [18.09853379121137, 59.301011587691676],
                  [18.099864166882757, 59.301011587691676],
                  [18.099864166882757, 59.30164695649144]]], None, False),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.110914868024114, 59.30852570710506],
                  [18.110914868024114, 59.30794522935226],
                  [18.112695854810003, 59.30794522935226],
                  [18.112695854810003, 59.30852570710506]]], None, False),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.066261452344182, 59.307101876095324],
                  [18.066261452344182, 59.30677329124305],
                  [18.06681935181928, 59.30677329124305],
                  [18.06681935181928, 59.307101876095324]]], None, False),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.054513376858953, 59.30043098168759],
                  [18.054513376858953, 59.3000256470941],
                  [18.055232208874944, 59.3000256470941],
                  [18.055232208874944, 59.30043098168759]]], None, False),
            {
              "landcover": 2,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.075874489453557, 59.306357079206215],
                  [18.075874489453557, 59.30611611201477],
                  [18.076260727551702, 59.30611611201477],
                  [18.076260727551702, 59.306357079206215]]], None, False),
            {
              "landcover": 2,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.0802947699101, 59.29939024801239],
                  [18.0802947699101, 59.29859050471372],
                  [18.08209721436811, 59.29859050471372],
                  [18.08209721436811, 59.29939024801239]]], None, False),
            {
              "landcover": 2,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.013330830686225, 59.35244870217859],
                  [18.013330830686225, 59.347646506653334],
                  [18.01674260055317, 59.347646506653334],
                  [18.01674260055317, 59.35244870217859]]], None, False),
            {
              "landcover": 2,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.028608693235054, 59.34636653580566],
                  [18.028608693235054, 59.34530532681132],
                  [18.031076325528755, 59.34530532681132],
                  [18.031076325528755, 59.34636653580566]]], None, False),
            {
              "landcover": 2,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.064890368250243, 59.33083229652714],
                  [18.064890368250243, 59.329803438513316],
                  [18.066907389429442, 59.329803438513316],
                  [18.066907389429442, 59.33083229652714]]], None, False),
            {
              "landcover": 2,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.067894442346923, 59.32531550203633],
                  [18.067894442346923, 59.32406753145792],
                  [18.071027262476317, 59.32406753145792],
                  [18.071027262476317, 59.32531550203633]]], None, False),
            {
              "landcover": 2,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.102990455395624, 59.33686060020329],
                  [18.102990455395624, 59.33613834195931],
                  [18.104921645886346, 59.33613834195931],
                  [18.104921645886346, 59.33686060020329]]], None, False),
            {
              "landcover": 2,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.1086532678983, 59.3073255536445],
                  [18.1086532678983, 59.30637265509399],
                  [18.11120673088048, 59.30637265509399],
                  [18.11120673088048, 59.3073255536445]]], None, False),
            {
              "landcover": 2,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.120450899216287, 59.30631788999817],
                  [18.120450899216287, 59.30538686987756],
                  [18.122489378067606, 59.30538686987756],
                  [18.122489378067606, 59.30631788999817]]], None, False),
            {
              "landcover": 2,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.12557928285276, 59.306186453408486],
                  [18.12557928285276, 59.305047315020744],
                  [18.12787525376951, 59.305047315020744],
                  [18.12787525376951, 59.306186453408486]]], None, False),
            {
              "landcover": 2,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.106975481125467, 59.31198013407072],
                  [18.106975481125467, 59.311377812491756],
                  [18.108541891190164, 59.311377812491756],
                  [18.108541891190164, 59.31198013407072]]], None, False),
            {
              "landcover": 2,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.096783086868875, 59.316009938495576],
                  [18.096783086868875, 59.315002532165614],
                  [18.099358007523172, 59.315002532165614],
                  [18.099358007523172, 59.316009938495576]]], None, False),
            {
              "landcover": 2,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.036620268257007, 59.413166157244326],
                  [18.036620268257007, 59.412352711824404],
                  [18.03825105133806, 59.412352711824404],
                  [18.03825105133806, 59.413166157244326]]], None, False),
            {
              "landcover": 2,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.036180385978565, 59.41186135948843],
                  [18.036180385978565, 59.410960528358245],
                  [18.038122305305347, 59.410960528358245],
                  [18.038122305305347, 59.41186135948843]]], None, False),
            {
              "landcover": 2,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.018676289947376, 59.40400148298783],
                  [18.018676289947376, 59.403870424087835],
                  [18.01906252804552, 59.403870424087835],
                  [18.01906252804552, 59.40400148298783]]], None, False),
            {
              "landcover": 2,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.017640957267627, 59.40342263568758],
                  [18.017640957267627, 59.40324515694002],
                  [18.018005737693652, 59.40324515694002],
                  [18.018005737693652, 59.40342263568758]]], None, False),
            {
              "landcover": 2,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01033138367726, 59.4058735853814],
                  [18.01033138367726, 59.40572478710592],
                  [18.010580829115646, 59.40572478710592],
                  [18.010580829115646, 59.4058735853814]]], None, False),
            {
              "landcover": 2,
              "system:index": "24"
            })])
# Class 3 - Bare Field:
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[17.982329285031298, 59.490137777722076],
                  [17.982329285031298, 59.48892854715184],
                  [17.983960068112353, 59.48892854715184],
                  [17.983960068112353, 59.490137777722076]]], None, False),
            {
              "landcover": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.9967917560396, 59.48749048685693],
                  [17.9967917560396, 59.48559476834364],
                  [18.00129786718462, 59.48559476834364],
                  [18.00129786718462, 59.48749048685693]]], None, False),
            {
              "landcover": 3,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.001619732266406, 59.495965443814505],
                  [18.001619732266406, 59.494898021360086],
                  [18.003701126461962, 59.494898021360086],
                  [18.003701126461962, 59.495965443814505]]], None, False),
            {
              "landcover": 3,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.041058933621386, 59.49537727643328],
                  [18.041058933621386, 59.493558249490505],
                  [18.043269073849658, 59.493558249490505],
                  [18.043269073849658, 59.49537727643328]]], None, False),
            {
              "landcover": 3,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.88718596685503, 59.49958135881831],
                  [17.88718596685503, 59.49840513982099],
                  [17.889589226132372, 59.49840513982099],
                  [17.889589226132372, 59.49958135881831]]], None, False),
            {
              "landcover": 3,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.887774834533488, 59.591804275916374],
                  [17.887774834533488, 59.59067471774544],
                  [17.88949144830302, 59.59067471774544],
                  [17.88949144830302, 59.591804275916374]]], None, False),
            {
              "landcover": 3,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.90395391931132, 59.58652543804897],
                  [17.90395391931132, 59.58411384133924],
                  [17.905756363769328, 59.58411384133924],
                  [17.905756363769328, 59.58652543804897]]], None, False),
            {
              "landcover": 3,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.104920177676632, 59.342211009243925],
                  [18.104920177676632, 59.34031808274802],
                  [18.10751655600305, 59.34031808274802],
                  [18.10751655600305, 59.342211009243925]]], None, False),
            {
              "landcover": 3,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.859531383896055, 59.46593785221798],
                  [17.859531383896055, 59.46515292925379],
                  [17.861462574386778, 59.46515292925379],
                  [17.861462574386778, 59.46593785221798]]], None, False),
            {
              "landcover": 3,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.98619808294641, 59.21044807923359],
                  [17.98619808294641, 59.209239830315425],
                  [17.988987580321897, 59.209239830315425],
                  [17.988987580321897, 59.21044807923359]]], None, False),
            {
              "landcover": 3,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.955642357848753, 59.213589326313965],
                  [17.955642357848753, 59.21288641500727],
                  [17.957101479552854, 59.21288641500727],
                  [17.957101479552854, 59.213589326313965]]], None, False),
            {
              "landcover": 3,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.05248183016572, 59.21055217055643],
                  [18.05248183016572, 59.20967345099012],
                  [18.054370105312206, 59.20967345099012],
                  [18.054370105312206, 59.21055217055643]]], None, False),
            {
              "landcover": 3,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.032912433193065, 59.22640917081584],
                  [18.032912433193065, 59.22601393348114],
                  [18.034199893520213, 59.22601393348114],
                  [18.034199893520213, 59.22640917081584]]], None, False),
            {
              "landcover": 3,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.06801718477998, 59.21661471918568],
                  [18.06801718477998, 59.21556044024356],
                  [18.070849597499706, 59.21556044024356],
                  [18.070849597499706, 59.21661471918568]]], None, False),
            {
              "landcover": 3,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.108980596171627, 59.337238095878476],
                  [18.108980596171627, 59.33472109681369],
                  [18.117284715281734, 59.33472109681369],
                  [18.117284715281734, 59.337238095878476]]], None, False),
            {
              "landcover": 3,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.91700887873294, 59.4046786873661],
                  [17.91700887873294, 59.40304044742301],
                  [17.919669630075713, 59.40304044742301],
                  [17.919669630075713, 59.4046786873661]]], None, False),
            {
              "landcover": 3,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.909155370737334, 59.401686109258584],
                  [17.909155370737334, 59.40055017098978],
                  [17.911687376047393, 59.40055017098978],
                  [17.911687376047393, 59.401686109258584]]], None, False),
            {
              "landcover": 3,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.889414312387725, 59.414157080424516],
                  [17.889414312387725, 59.41262848674568],
                  [17.892203809763213, 59.41262848674568],
                  [17.892203809763213, 59.414157080424516]]], None, False),
            {
              "landcover": 3,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.901322141057115, 59.40799004051288],
                  [17.901322141057115, 59.40589328415362],
                  [17.905742421513658, 59.40589328415362],
                  [17.905742421513658, 59.40799004051288]]], None, False),
            {
              "landcover": 3,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.917372479802232, 59.40185229387586],
                  [17.917372479802232, 59.40058528957054],
                  [17.919303670292955, 59.40058528957054],
                  [17.919303670292955, 59.40185229387586]]], None, False),
            {
              "landcover": 3,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.025706080799527, 59.39583357416297],
                  [18.025706080799527, 59.39505794802263],
                  [18.027014998798794, 59.39505794802263],
                  [18.027014998798794, 59.39583357416297]]], None, False),
            {
              "landcover": 3,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.016447095280117, 59.402343764127906],
                  [18.016447095280117, 59.40198879439989],
                  [18.017015723591275, 59.40198879439989],
                  [18.017015723591275, 59.402343764127906]]], None, False),
            {
              "landcover": 3,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.036756781940884, 59.409322259739454],
                  [18.036756781940884, 59.40821933808673],
                  [18.038033513431973, 59.40821933808673],
                  [18.038033513431973, 59.409322259739454]]], None, False),
            {
              "landcover": 3,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.04106977403683, 59.40983002842758],
                  [18.04106977403683, 59.408759883383944],
                  [18.043204812412686, 59.408759883383944],
                  [18.043204812412686, 59.40983002842758]]], None, False),
            {
              "landcover": 3,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.030555514698452, 59.411478859202056],
                  [18.030555514698452, 59.41109668702971],
                  [18.031531838779873, 59.41109668702971],
                  [18.031531838779873, 59.411478859202056]]], None, False),
            {
              "landcover": 3,
              "system:index": "24"
            })])

Map.addLayer(water, {}, "water")
Map.addLayer(vegetation, {}, "vegetation")
Map.addLayer(urban, {}, "urban")
Map.addLayer(bare, {}, "bare")
Map

In [10]:
# Import User Labelled Polygons from Supervised Classification for Accuracy Assessment:
# 0 - Urban
urban = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.052726907332364, 59.33283313087029],
                  [18.052726907332364, 59.33097251700143],
                  [18.056632203658047, 59.33097251700143],
                  [18.056632203658047, 59.33283313087029]]], None, False),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.123178256624882, 59.34183084125545],
                  [18.123178256624882, 59.33977376018854],
                  [18.1297013889491, 59.33977376018854],
                  [18.1297013889491, 59.34183084125545]]], None, False),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.08752600541425, 59.302238496085714],
                  [18.08752600541425, 59.30166886554873],
                  [18.088727635052923, 59.30166886554873],
                  [18.088727635052923, 59.302238496085714]]], None, False),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.091452759412054, 59.30506459883667],
                  [18.091452759412054, 59.304681226570594],
                  [18.091903370526556, 59.304681226570594],
                  [18.091903370526556, 59.30506459883667]]], None, False),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.09853379121137, 59.30164695649144],
                  [18.09853379121137, 59.301011587691676],
                  [18.099864166882757, 59.301011587691676],
                  [18.099864166882757, 59.30164695649144]]], None, False),
            {
              "landcover": 0,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.110914868024114, 59.30852570710506],
                  [18.110914868024114, 59.30794522935226],
                  [18.112695854810003, 59.30794522935226],
                  [18.112695854810003, 59.30852570710506]]], None, False),
            {
              "landcover": 0,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.066261452344182, 59.307101876095324],
                  [18.066261452344182, 59.30677329124305],
                  [18.06681935181928, 59.30677329124305],
                  [18.06681935181928, 59.307101876095324]]], None, False),
            {
              "landcover": 0,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.054513376858953, 59.30043098168759],
                  [18.054513376858953, 59.3000256470941],
                  [18.055232208874944, 59.3000256470941],
                  [18.055232208874944, 59.30043098168759]]], None, False),
            {
              "landcover": 0,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.075874489453557, 59.306357079206215],
                  [18.075874489453557, 59.30611611201477],
                  [18.076260727551702, 59.30611611201477],
                  [18.076260727551702, 59.306357079206215]]], None, False),
            {
              "landcover": 0,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.0802947699101, 59.29939024801239],
                  [18.0802947699101, 59.29859050471372],
                  [18.08209721436811, 59.29859050471372],
                  [18.08209721436811, 59.29939024801239]]], None, False),
            {
              "landcover": 0,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.013330830686225, 59.35244870217859],
                  [18.013330830686225, 59.347646506653334],
                  [18.01674260055317, 59.347646506653334],
                  [18.01674260055317, 59.35244870217859]]], None, False),
            {
              "landcover": 0,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.028608693235054, 59.34636653580566],
                  [18.028608693235054, 59.34530532681132],
                  [18.031076325528755, 59.34530532681132],
                  [18.031076325528755, 59.34636653580566]]], None, False),
            {
              "landcover": 0,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.064890368250243, 59.33083229652714],
                  [18.064890368250243, 59.329803438513316],
                  [18.066907389429442, 59.329803438513316],
                  [18.066907389429442, 59.33083229652714]]], None, False),
            {
              "landcover": 0,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.067894442346923, 59.32531550203633],
                  [18.067894442346923, 59.32406753145792],
                  [18.071027262476317, 59.32406753145792],
                  [18.071027262476317, 59.32531550203633]]], None, False),
            {
              "landcover": 0,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.102990455395624, 59.33686060020329],
                  [18.102990455395624, 59.33613834195931],
                  [18.104921645886346, 59.33613834195931],
                  [18.104921645886346, 59.33686060020329]]], None, False),
            {
              "landcover": 0,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.1086532678983, 59.3073255536445],
                  [18.1086532678983, 59.30637265509399],
                  [18.11120673088048, 59.30637265509399],
                  [18.11120673088048, 59.3073255536445]]], None, False),
            {
              "landcover": 0,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.120450899216287, 59.30631788999817],
                  [18.120450899216287, 59.30538686987756],
                  [18.122489378067606, 59.30538686987756],
                  [18.122489378067606, 59.30631788999817]]], None, False),
            {
              "landcover": 0,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.12557928285276, 59.306186453408486],
                  [18.12557928285276, 59.305047315020744],
                  [18.12787525376951, 59.305047315020744],
                  [18.12787525376951, 59.306186453408486]]], None, False),
            {
              "landcover": 0,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.106975481125467, 59.31198013407072],
                  [18.106975481125467, 59.311377812491756],
                  [18.108541891190164, 59.311377812491756],
                  [18.108541891190164, 59.31198013407072]]], None, False),
            {
              "landcover": 0,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.096783086868875, 59.316009938495576],
                  [18.096783086868875, 59.315002532165614],
                  [18.099358007523172, 59.315002532165614],
                  [18.099358007523172, 59.316009938495576]]], None, False),
            {
              "landcover": 0,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.036620268257007, 59.413166157244326],
                  [18.036620268257007, 59.412352711824404],
                  [18.03825105133806, 59.412352711824404],
                  [18.03825105133806, 59.413166157244326]]], None, False),
            {
              "landcover": 0,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.036180385978565, 59.41186135948843],
                  [18.036180385978565, 59.410960528358245],
                  [18.038122305305347, 59.410960528358245],
                  [18.038122305305347, 59.41186135948843]]], None, False),
            {
              "landcover": 0,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.018676289947376, 59.40400148298783],
                  [18.018676289947376, 59.403870424087835],
                  [18.01906252804552, 59.403870424087835],
                  [18.01906252804552, 59.40400148298783]]], None, False),
            {
              "landcover": 0,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.017640957267627, 59.40342263568758],
                  [18.017640957267627, 59.40324515694002],
                  [18.018005737693652, 59.40324515694002],
                  [18.018005737693652, 59.40342263568758]]], None, False),
            {
              "landcover": 0,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01033138367726, 59.4058735853814],
                  [18.01033138367726, 59.40572478710592],
                  [18.010580829115646, 59.40572478710592],
                  [18.010580829115646, 59.4058735853814]]], None, False),
            {
              "landcover": 0,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.11234152220315, 59.372992776659515],
                  [18.11234152220315, 59.37257190943273],
                  [18.112953065858544, 59.37257190943273],
                  [18.112953065858544, 59.372992776659515]]], None, False),
            {
              "landcover": 0,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.108804353600288, 59.35238505130048],
                  [18.108804353600288, 59.3508536703185],
                  [18.114683755760932, 59.3508536703185],
                  [18.114683755760932, 59.35238505130048]]], None, False),
            {
              "landcover": 0,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.100156911736274, 59.35345697688813],
                  [18.100156911736274, 59.35190376757798],
                  [18.105907567864204, 59.35190376757798],
                  [18.105907567864204, 59.35345697688813]]], None, False),
            {
              "landcover": 0,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.096122869377876, 59.36079547656358],
                  [18.096122869377876, 59.35911137101829],
                  [18.098333009606147, 59.35911137101829],
                  [18.098333009606147, 59.36079547656358]]], None, False),
            {
              "landcover": 0,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.09073699367597, 59.359669103538366],
                  [18.09073699367597, 59.35802868760927],
                  [18.094406255608344, 59.35802868760927],
                  [18.094406255608344, 59.359669103538366]]], None, False),
            {
              "landcover": 0,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.08848393810346, 59.357645912484216],
                  [18.08848393810346, 59.35544760604216],
                  [18.090887197380805, 59.35544760604216],
                  [18.090887197380805, 59.357645912484216]]], None, False),
            {
              "landcover": 0,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.105585702782417, 59.35618039067538],
                  [18.105585702782417, 59.35414605403671],
                  [18.10852540386274, 59.35414605403671],
                  [18.10852540386274, 59.35618039067538]]], None, False),
            {
              "landcover": 0,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.102109559899116, 59.35873954424672],
                  [18.102109559899116, 59.35708814671702],
                  [18.105156549340034, 59.35708814671702],
                  [18.105156549340034, 59.35873954424672]]], None, False),
            {
              "landcover": 0,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.093333372002387, 59.3566288033207],
                  [18.093333372002387, 59.35618039067568],
                  [18.094191678887153, 59.35618039067568],
                  [18.094191678887153, 59.3566288033207]]], None, False),
            {
              "landcover": 0,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.090715536003852, 59.35133496892889],
                  [18.090715536003852, 59.35014264852597],
                  [18.095715173607612, 59.35014264852597],
                  [18.095715173607612, 59.35133496892889]]], None, False),
            {
              "landcover": 0,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.098676332360053, 59.35055832461343],
                  [18.098676332360053, 59.34933315943475],
                  [18.101658948784614, 59.34933315943475],
                  [18.101658948784614, 59.35055832461343]]], None, False),
            {
              "landcover": 0,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.10897601497724, 59.34908155754267],
                  [18.10897601497724, 59.346959276090146],
                  [18.11298859966352, 59.346959276090146],
                  [18.11298859966352, 59.34908155754267]]], None, False),
            {
              "landcover": 0,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.113632329827094, 59.34688269644495],
                  [18.113632329827094, 59.34516507902331],
                  [18.11601413143232, 59.34516507902331],
                  [18.11601413143232, 59.34688269644495]]], None, False),
            {
              "landcover": 0,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.113546499138618, 59.34366619538469],
                  [18.113546499138618, 59.33977097675323],
                  [18.118631967430854, 59.33977097675323],
                  [18.118631967430854, 59.34366619538469]]], None, False),
            {
              "landcover": 0,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.125326761132026, 59.34627005306504],
                  [18.125326761132026, 59.34502285203013],
                  [18.1289960230644, 59.34502285203013],
                  [18.1289960230644, 59.34627005306504]]], None, False),
            {
              "landcover": 0,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.132171758538032, 59.341795233202745],
                  [18.132171758538032, 59.33898312541352],
                  [18.13620580089643, 59.33898312541352],
                  [18.13620580089643, 59.341795233202745]]], None, False),
            {
              "landcover": 0,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.081445821648384, 59.334551121116775],
                  [18.081445821648384, 59.33190258576638],
                  [18.08702481639936, 59.33190258576638],
                  [18.08702481639936, 59.334551121116775]]], None, False),
            {
              "landcover": 0,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.076102861290718, 59.330665802264626],
                  [18.076102861290718, 59.32895831035639],
                  [18.077883848076606, 59.32895831035639],
                  [18.077883848076606, 59.330665802264626]]], None, False),
            {
              "landcover": 0,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.06681168926313, 59.33242793150725],
                  [18.06681168926313, 59.33050162407892],
                  [18.070995935326362, 59.33050162407892],
                  [18.070995935326362, 59.33242793150725]]], None, False),
            {
              "landcover": 0,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.057627805596137, 59.33543756823588],
                  [18.057627805596137, 59.332734379438314],
                  [18.063142427330757, 59.332734379438314],
                  [18.063142427330757, 59.33543756823588]]], None, False),
            {
              "landcover": 0,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.066918977623725, 59.336367765649314],
                  [18.066918977623725, 59.333325378356406],
                  [18.07202590358808, 59.333325378356406],
                  [18.07202590358808, 59.336367765649314]]], None, False),
            {
              "landcover": 0,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.074965604668403, 59.33634587894395],
                  [18.074965604668403, 59.334748111364725],
                  [18.07655347240522, 59.334748111364725],
                  [18.07655347240522, 59.33634587894395]]], None, False),
            {
              "landcover": 0,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.061940797692085, 59.32557590961766],
                  [18.061940797692085, 59.324327948601535],
                  [18.06427968395307, 59.324327948601535],
                  [18.06427968395307, 59.32557590961766]]], None, False),
            {
              "landcover": 0,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.070116170769477, 59.327053698925624],
                  [18.070116170769477, 59.32536791929732],
                  [18.072605260735298, 59.32536791929732],
                  [18.072605260735298, 59.327053698925624]]], None, False),
            {
              "landcover": 0,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.073688873177314, 59.32490814670384],
                  [18.073688873177314, 59.32308541557434],
                  [18.07600630176618, 59.32308541557434],
                  [18.07600630176618, 59.32490814670384]]], None, False),
            {
              "landcover": 0,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.014933906145004, 59.298658811950276],
                  [18.014933906145004, 59.291843811956106],
                  [18.02853807026854, 59.291843811956106],
                  [18.02853807026854, 59.298658811950276]]], None, False),
            {
              "landcover": 0,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.02896722371092, 59.29605130436041],
                  [18.02896722371092, 59.29322445274832],
                  [18.032615027971175, 59.29322445274832],
                  [18.032615027971175, 59.29605130436041]]], None, False),
            {
              "landcover": 0,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.007123313493636, 59.2935969970161],
                  [18.007123313493636, 59.29050694765425],
                  [18.01304563099852, 59.29050694765425],
                  [18.01304563099852, 59.2935969970161]]], None, False),
            {
              "landcover": 0,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.050762952358074, 59.287753021604054],
                  [18.050762952358074, 59.28615296911046],
                  [18.054153264552898, 59.28615296911046],
                  [18.054153264552898, 59.287753021604054]]], None, False),
            {
              "landcover": 0,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.04408961632902, 59.292004850361536],
                  [18.04408961632902, 59.290043376807],
                  [18.047994912654705, 59.290043376807],
                  [18.047994912654705, 59.292004850361536]]], None, False),
            {
              "landcover": 0,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.054196179897136, 59.28957216799478],
                  [18.054196179897136, 59.287621513277216],
                  [18.057264627010174, 59.287621513277216],
                  [18.057264627010174, 59.28957216799478]]], None, False),
            {
              "landcover": 0,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.0106585631674, 59.28036585000463],
                  [18.0106585631674, 59.27793233568166],
                  [18.019499124080486, 59.27793233568166],
                  [18.019499124080486, 59.28036585000463]]], None, False),
            {
              "landcover": 0,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.001753629237957, 59.272626232999606],
                  [18.001753629237957, 59.2706088200282],
                  [18.008319676906414, 59.2706088200282],
                  [18.008319676906414, 59.272626232999606]]], None, False),
            {
              "landcover": 0,
              "system:index": "57"
            })])
# 1 - Water
water = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[18.1287605285643, 59.420668377041466],
                  [18.1287605285643, 59.41855057178141],
                  [18.132880401611175, 59.41855057178141],
                  [18.132880401611175, 59.420668377041466]]], None, False),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.155077293266338, 59.390922117625365],
                  [18.155077293266338, 59.388518363011805],
                  [18.160570457328838, 59.388518363011805],
                  [18.160570457328838, 59.390922117625365]]], None, False),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.101690605033916, 59.36862621806919],
                  [18.101690605033916, 59.36512749188252],
                  [18.107784583915752, 59.36512749188252],
                  [18.107784583915752, 59.36862621806919]]], None, False),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.078485317078293, 59.322705646967805],
                  [18.078485317078293, 59.32136999522701],
                  [18.0823047827155, 59.32136999522701],
                  [18.0823047827155, 59.322705646967805]]], None, False),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.056233263540545, 59.32401824428743],
                  [18.056233263540545, 59.322047668267054],
                  [18.059752321768084, 59.322047668267054],
                  [18.059752321768084, 59.32401824428743]]], None, False),
            {
              "landcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.990464791118104, 59.330316839397135],
                  [17.990464791118104, 59.32834662857648],
                  [17.994155510722596, 59.32834662857648],
                  [17.994155510722596, 59.330316839397135]]], None, False),
            {
              "landcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.98780403977533, 59.34467403662359],
                  [17.98780403977533, 59.34252959696393],
                  [17.991494759379822, 59.34252959696393],
                  [17.991494759379822, 59.34467403662359]]], None, False),
            {
              "landcover": 1,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.25405792757162, 59.37983915870227],
                  [18.25405792757162, 59.37826529692743],
                  [18.25731949373373, 59.37826529692743],
                  [18.25731949373373, 59.37983915870227]]], None, False),
            {
              "landcover": 1,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.789414237013364, 59.3573267225729],
                  [17.789414237013364, 59.35277679343631],
                  [17.797653983107114, 59.35277679343631],
                  [17.797653983107114, 59.3573267225729]]], None, False),
            {
              "landcover": 1,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.732422659864927, 59.4209617222526],
                  [17.732422659864927, 59.41711904193939],
                  [17.736542532911802, 59.41711904193939],
                  [17.736542532911802, 59.4209617222526]]], None, False),
            {
              "landcover": 1,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.76881487177899, 59.44645209758956],
                  [17.76881487177899, 59.4440086473052],
                  [17.779114554396177, 59.4440086473052],
                  [17.779114554396177, 59.44645209758956]]], None, False),
            {
              "landcover": 1,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.316071341505552, 59.30794491711008],
                  [18.316071341505552, 59.30373889551123],
                  [18.324311087599302, 59.30373889551123],
                  [18.324311087599302, 59.30794491711008]]], None, False),
            {
              "landcover": 1,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.161232779493833, 59.32695314970951],
                  [18.161232779493833, 59.32485137940307],
                  [18.166382620802427, 59.32485137940307],
                  [18.166382620802427, 59.32695314970951]]], None, False),
            {
              "landcover": 1,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.280552863119695, 59.38369091578032],
                  [18.280552863119695, 59.37980028875377],
                  [18.289565085409734, 59.37980028875377],
                  [18.289565085409734, 59.38369091578032]]], None, False),
            {
              "landcover": 1,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.033239451895742, 59.50996377764133],
                  [18.033239451895742, 59.50482454456448],
                  [18.0418225207434, 59.50482454456448],
                  [18.0418225207434, 59.50996377764133]]], None, False),
            {
              "landcover": 1,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.09538833818113, 59.31835313775008],
                  [18.09538833818113, 59.31728010393107],
                  [18.098478242966287, 59.31728010393107],
                  [18.098478242966287, 59.31835313775008]]], None, False),
            {
              "landcover": 1,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.09163324556028, 59.32151729537798],
                  [18.09163324556028, 59.31990788176361],
                  [18.09493772706663, 59.31990788176361],
                  [18.09493772706663, 59.32151729537798]]], None, False),
            {
              "landcover": 1,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.04970794169684, 59.30631471939471],
                  [18.04970794169684, 59.30451837520289],
                  [18.05350594966193, 59.30451837520289],
                  [18.05350594966193, 59.30631471939471]]], None, False),
            {
              "landcover": 1,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.033850722000796, 59.30993997876862],
                  [18.033850722000796, 59.30795763406247],
                  [18.037112288162906, 59.30795763406247],
                  [18.037112288162906, 59.30993997876862]]], None, False),
            {
              "landcover": 1,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.01400237529059, 59.312108378207],
                  [18.01400237529059, 59.31138559375791],
                  [18.01655583827277, 59.31138559375791],
                  [18.01655583827277, 59.312108378207]]], None, False),
            {
              "landcover": 1,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.00598936987815, 59.399749007250186],
                  [18.00598936987815, 59.39735680230275],
                  [18.009744462499, 59.39735680230275],
                  [18.009744462499, 59.399749007250186]]], None, False),
            {
              "landcover": 1,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.991934594640114, 59.415146831337566],
                  [17.991934594640114, 59.41191496074184],
                  [17.99704152060447, 59.41191496074184],
                  [17.99704152060447, 59.415146831337566]]], None, False),
            {
              "landcover": 1,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.05465537024436, 59.40777640587995],
                  [18.05465537024436, 59.40706111625854],
                  [18.05623250914512, 59.40706111625854],
                  [18.05623250914512, 59.40777640587995]]], None, False),
            {
              "landcover": 1,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.060865223528232, 59.37764527486218],
                  [18.060865223528232, 59.374934514359545],
                  [18.067259609819736, 59.374934514359545],
                  [18.067259609819736, 59.37764527486218]]], None, False),
            {
              "landcover": 1,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.022541821123447, 59.37646480901623],
                  [18.022541821123447, 59.37438796382078],
                  [18.025545895220127, 59.37438796382078],
                  [18.025545895220127, 59.37646480901623]]], None, False),
            {
              "landcover": 1,
              "system:index": "24"
            })])
# 2 - Forest/Veg
vegetation = ee.FeatureCollection(
    [ee.Feature(
        ee.Geometry.Polygon(
            [[[18.01369561111225, 59.34494428938679],
              [18.01369561111225, 59.344594188824566],
              [18.01425351058735, 59.344594188824566],
              [18.01425351058735, 59.34494428938679]]], None, False),
        {
          "landcover": 2,
          "system:index": "0"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.014038933866157, 59.341924553350076],
              [18.014038933866157, 59.341661954918315],
              [18.0149830714394, 59.341661954918315],
              [18.0149830714394, 59.341924553350076]]], None, False),
        {
          "landcover": 2,
          "system:index": "1"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.02884909415171, 59.36211740667467],
              [18.02884909415171, 59.36028026804388],
              [18.032239406346534, 59.36028026804388],
              [18.032239406346534, 59.36211740667467]]], None, False),
        {
          "landcover": 2,
          "system:index": "2"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.060434787511085, 59.36064661017352],
              [18.060434787511085, 59.35982096784676],
              [18.061969011067603, 59.35982096784676],
              [18.061969011067603, 59.36064661017352]]], None, False),
        {
          "landcover": 2,
          "system:index": "3"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.063610522984717, 59.36341865303429],
              [18.063610522984717, 59.362489196440386],
              [18.066153257130836, 59.362489196440386],
              [18.066153257130836, 59.36341865303429]]], None, False),
        {
          "landcover": 2,
          "system:index": "4"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.049169509648536, 59.36349519538389],
              [18.049169509648536, 59.36255480597621],
              [18.051122157811378, 59.36255480597621],
              [18.051122157811378, 59.36349519538389]]], None, False),
        {
          "landcover": 2,
          "system:index": "5"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.071378200291846, 59.35682442829472],
              [18.071378200291846, 59.3545713812703],
              [18.07674261832163, 59.3545713812703],
              [18.07674261832163, 59.35682442829472]]], None, False),
        {
          "landcover": 2,
          "system:index": "6"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.039706676243995, 59.35710878282482],
              [18.039706676243995, 59.355818231598626],
              [18.04232451224253, 59.355818231598626],
              [18.04232451224253, 59.35710878282482]]], None, False),
        {
          "landcover": 2,
          "system:index": "7"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.078321700064755, 59.364546150150915],
              [18.078321700064755, 59.363693266342196],
              [18.081046824423886, 59.363693266342196],
              [18.081046824423886, 59.364546150150915]]], None, False),
        {
          "landcover": 2,
          "system:index": "8"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.082505946127988, 59.36660173071935],
              [18.082505946127988, 59.36565049348929],
              [18.08473754402838, 59.36565049348929],
              [18.08473754402838, 59.36660173071935]]], None, False),
        {
          "landcover": 2,
          "system:index": "9"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.090574030844785, 59.36484137416718],
              [18.090574030844785, 59.36314653467683],
              [18.093513731925107, 59.36314653467683],
              [18.093513731925107, 59.36484137416718]]], None, False),
        {
          "landcover": 2,
          "system:index": "10"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.121429990560365, 59.328055748769714],
              [18.121429990560365, 59.32693923570562],
              [18.123361181051088, 59.32693923570562],
              [18.123361181051088, 59.328055748769714]]], None, False),
        {
          "landcover": 2,
          "system:index": "11"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.13576371553595, 59.32540670706443],
              [18.13576371553595, 59.32398358287265],
              [18.13825280550177, 59.32398358287265],
              [18.13825280550177, 59.32540670706443]]], None, False),
        {
          "landcover": 2,
          "system:index": "12"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.048286438822153, 59.31351383988241],
              [18.048286438822153, 59.31245160437228],
              [18.050754071115854, 59.31245160437228],
              [18.050754071115854, 59.31351383988241]]], None, False),
        {
          "landcover": 2,
          "system:index": "13"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.12301317314356, 59.338600584300885],
              [18.12301317314356, 59.33581009907216],
              [18.128549252550297, 59.33581009907216],
              [18.128549252550297, 59.338600584300885]]], None, False),
        {
          "landcover": 2,
          "system:index": "14"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.909927846933623, 59.40869750046596],
              [17.909927846933623, 59.406819201511915],
              [17.913446905161162, 59.406819201511915],
              [17.913446905161162, 59.40869750046596]]], None, False),
        {
          "landcover": 2,
          "system:index": "15"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.912331106210967, 59.399588962709345],
              [17.912331106210967, 59.39823448657892],
              [17.915592672373077, 59.39823448657892],
              [17.915592672373077, 59.399588962709345]]], None, False),
        {
          "landcover": 2,
          "system:index": "16"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.88470670100653, 59.40917671266396],
              [17.88470670100653, 59.4071892351965],
              [17.88865491267645, 59.4071892351965],
              [17.88865491267645, 59.40917671266396]]], None, False),
        {
          "landcover": 2,
          "system:index": "17"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.979135726798837, 59.411884660487154],
              [17.979135726798837, 59.410661709009055],
              [17.98059484850294, 59.410661709009055],
              [17.98059484850294, 59.411884660487154]]], None, False),
        {
          "landcover": 2,
          "system:index": "18"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.985658859123056, 59.409657108689025],
              [17.985658859123056, 59.40822659376949],
              [17.988405441154306, 59.40822659376949],
              [17.988405441154306, 59.409657108689025]]], None, False),
        {
          "landcover": 2,
          "system:index": "19"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.014604199818365, 59.407413737428996],
              [18.014604199818365, 59.40660015384685],
              [18.016084779194586, 59.40660015384685],
              [18.016084779194586, 59.407413737428996]]], None, False),
        {
          "landcover": 2,
          "system:index": "20"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.010623801640264, 59.405022073609665],
              [18.010623801640264, 59.40459068248083],
              [18.011525023869268, 59.40459068248083],
              [18.011525023869268, 59.405022073609665]]], None, False),
        {
          "landcover": 2,
          "system:index": "21"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.99300747824607, 59.40285097587157],
              [17.99300747824607, 59.401540319550485],
              [17.996075925359108, 59.401540319550485],
              [17.996075925359108, 59.40285097587157]]], None, False),
        {
          "landcover": 2,
          "system:index": "22"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.995432195195534, 59.39848192429468],
              [17.995432195195534, 59.397040013642794],
              [17.997642335423805, 59.397040013642794],
              [17.997642335423805, 59.39848192429468]]], None, False),
        {
          "landcover": 2,
          "system:index": "23"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.988222417363502, 59.39558711737275],
              [17.988222417363502, 59.39446189926777],
              [17.990196523198463, 59.39446189926777],
              [17.990196523198463, 59.39558711737275]]], None, False),
        {
          "landcover": 2,
          "system:index": "24"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.12486536027203, 59.32542601751094],
              [18.12486536027203, 59.32542601751094],
              [18.12486536027203, 59.32542601751094],
              [18.12486536027203, 59.32542601751094]]], None, False),
        {
          "landcover": 2,
          "system:index": "25"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.19188899993881, 59.401253844064165],
              [18.19188899993881, 59.39686273883423],
              [18.205235671996917, 59.39686273883423],
              [18.205235671996917, 59.401253844064165]]], None, False),
        {
          "landcover": 2,
          "system:index": "26"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.17871398925766, 59.401712583422345],
              [18.17871398925766, 59.39852312413049],
              [18.187125396728362, 59.39852312413049],
              [18.187125396728362, 59.401712583422345]]], None, False),
        {
          "landcover": 2,
          "system:index": "27"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.220341873168792, 59.40096985944426],
              [18.220341873168792, 59.39917851698454],
              [18.226478767394866, 59.39917851698454],
              [18.226478767394866, 59.40096985944426]]], None, False),
        {
          "landcover": 2,
          "system:index": "28"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.24785060882553, 59.39948436264513],
              [18.24785060882553, 59.39507117924598],
              [18.2565624237059, 59.39507117924598],
              [18.2565624237059, 59.39948436264513]]], None, False),
        {
          "landcover": 2,
          "system:index": "29"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.239911270141448, 59.4133319089714],
              [18.239911270141448, 59.40961939989581],
              [18.246176910400237, 59.40961939989581],
              [18.246176910400237, 59.4133319089714]]], None, False),
        {
          "landcover": 2,
          "system:index": "30"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.204076957702483, 59.41189063024432],
              [18.204076957702483, 59.40651805097141],
              [18.21235961914047, 59.40651805097141],
              [18.21235961914047, 59.41189063024432]]], None, False),
        {
          "landcover": 2,
          "system:index": "31"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.208583068847503, 59.401996561818244],
              [18.208583068847503, 59.398392044039454],
              [18.214462471008147, 59.398392044039454],
              [18.214462471008147, 59.401996561818244]]], None, False),
        {
          "landcover": 2,
          "system:index": "32"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.254169893264617, 59.388264574351766],
              [18.254169893264617, 59.3861665971439],
              [18.25683064460739, 59.3861665971439],
              [18.25683064460739, 59.388264574351766]]], None, False),
        {
          "landcover": 2,
          "system:index": "33"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.227948617935027, 59.393312299964],
              [18.227948617935027, 59.390471681059225],
              [18.23258347511276, 59.390471681059225],
              [18.23258347511276, 59.393312299964]]], None, False),
        {
          "landcover": 2,
          "system:index": "34"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.21842141151413, 59.39471067105573],
              [18.21842141151413, 59.39156425496446],
              [18.223742914199676, 59.39156425496446],
              [18.223742914199676, 59.39471067105573]]], None, False),
        {
          "landcover": 2,
          "system:index": "35"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.268975687026824, 59.40128661136959],
              [18.268975687026824, 59.398534047451],
              [18.28433938026413, 59.398534047451],
              [18.28433938026413, 59.40128661136959]]], None, False),
        {
          "landcover": 2,
          "system:index": "36"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[18.268031549453582, 59.39805341812341],
              [18.268031549453582, 59.39643670581045],
              [18.270949792861785, 59.39643670581045],
              [18.270949792861785, 59.39805341812341]]], None, False),
        {
          "landcover": 2,
          "system:index": "37"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.896681380610467, 59.435167361517486],
              [17.896681380610467, 59.43180645095442],
              [17.90929849181652, 59.43180645095442],
              [17.90929849181652, 59.435167361517486]]], None, False),
        {
          "landcover": 2,
          "system:index": "38"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.910500121455193, 59.43377066000129],
              [17.910500121455193, 59.42813962235309],
              [17.915478301386834, 59.42813962235309],
              [17.915478301386834, 59.43377066000129]]], None, False),
        {
          "landcover": 2,
          "system:index": "39"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.87839944396496, 59.44175726897757],
              [17.87839944396496, 59.43486183798698],
              [17.88758332763195, 59.43486183798698],
              [17.88758332763195, 59.44175726897757]]], None, False),
        {
          "landcover": 2,
          "system:index": "40"
        }),
    ee.Feature(
        ee.Geometry.Polygon(
            [[[17.844858141287663, 59.35946616041206],
              [17.844858141287663, 59.35450089533381],
              [17.85468575511823, 59.35450089533381],
              [17.85468575511823, 59.35946616041206]]], None, False),
        {
          "landcover": 2,
          "system:index": "41"
        })]
)
# 3 - Bare Fields
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Polygon(
                [[[17.982329285031298, 59.490137777722076],
                  [17.982329285031298, 59.48892854715184],
                  [17.983960068112353, 59.48892854715184],
                  [17.983960068112353, 59.490137777722076]]], None, False),
            {
              "landcover": 3,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.9967917560396, 59.48749048685693],
                  [17.9967917560396, 59.48559476834364],
                  [18.00129786718462, 59.48559476834364],
                  [18.00129786718462, 59.48749048685693]]], None, False),
            {
              "landcover": 3,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.001619732266406, 59.495965443814505],
                  [18.001619732266406, 59.494898021360086],
                  [18.003701126461962, 59.494898021360086],
                  [18.003701126461962, 59.495965443814505]]], None, False),
            {
              "landcover": 3,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.041058933621386, 59.49537727643328],
                  [18.041058933621386, 59.493558249490505],
                  [18.043269073849658, 59.493558249490505],
                  [18.043269073849658, 59.49537727643328]]], None, False),
            {
              "landcover": 3,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.88718596685503, 59.49958135881831],
                  [17.88718596685503, 59.49840513982099],
                  [17.889589226132372, 59.49840513982099],
                  [17.889589226132372, 59.49958135881831]]], None, False),
            {
              "landcover": 3,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.887774834533488, 59.591804275916374],
                  [17.887774834533488, 59.59067471774544],
                  [17.88949144830302, 59.59067471774544],
                  [17.88949144830302, 59.591804275916374]]], None, False),
            {
              "landcover": 3,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.90395391931132, 59.58652543804897],
                  [17.90395391931132, 59.58411384133924],
                  [17.905756363769328, 59.58411384133924],
                  [17.905756363769328, 59.58652543804897]]], None, False),
            {
              "landcover": 3,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.104920177676632, 59.342211009243925],
                  [18.104920177676632, 59.34031808274802],
                  [18.10751655600305, 59.34031808274802],
                  [18.10751655600305, 59.342211009243925]]], None, False),
            {
              "landcover": 3,
              "system:index": "7"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.859531383896055, 59.46593785221798],
                  [17.859531383896055, 59.46515292925379],
                  [17.861462574386778, 59.46515292925379],
                  [17.861462574386778, 59.46593785221798]]], None, False),
            {
              "landcover": 3,
              "system:index": "8"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.98619808294641, 59.21044807923359],
                  [17.98619808294641, 59.209239830315425],
                  [17.988987580321897, 59.209239830315425],
                  [17.988987580321897, 59.21044807923359]]], None, False),
            {
              "landcover": 3,
              "system:index": "9"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.955642357848753, 59.213589326313965],
                  [17.955642357848753, 59.21288641500727],
                  [17.957101479552854, 59.21288641500727],
                  [17.957101479552854, 59.213589326313965]]], None, False),
            {
              "landcover": 3,
              "system:index": "10"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.05248183016572, 59.21055217055643],
                  [18.05248183016572, 59.20967345099012],
                  [18.054370105312206, 59.20967345099012],
                  [18.054370105312206, 59.21055217055643]]], None, False),
            {
              "landcover": 3,
              "system:index": "11"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.032912433193065, 59.22640917081584],
                  [18.032912433193065, 59.22601393348114],
                  [18.034199893520213, 59.22601393348114],
                  [18.034199893520213, 59.22640917081584]]], None, False),
            {
              "landcover": 3,
              "system:index": "12"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.06801718477998, 59.21661471918568],
                  [18.06801718477998, 59.21556044024356],
                  [18.070849597499706, 59.21556044024356],
                  [18.070849597499706, 59.21661471918568]]], None, False),
            {
              "landcover": 3,
              "system:index": "13"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.108980596171627, 59.337238095878476],
                  [18.108980596171627, 59.33472109681369],
                  [18.117284715281734, 59.33472109681369],
                  [18.117284715281734, 59.337238095878476]]], None, False),
            {
              "landcover": 3,
              "system:index": "14"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.91700887873294, 59.4046786873661],
                  [17.91700887873294, 59.40304044742301],
                  [17.919669630075713, 59.40304044742301],
                  [17.919669630075713, 59.4046786873661]]], None, False),
            {
              "landcover": 3,
              "system:index": "15"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.909155370737334, 59.401686109258584],
                  [17.909155370737334, 59.40055017098978],
                  [17.911687376047393, 59.40055017098978],
                  [17.911687376047393, 59.401686109258584]]], None, False),
            {
              "landcover": 3,
              "system:index": "16"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.889414312387725, 59.414157080424516],
                  [17.889414312387725, 59.41262848674568],
                  [17.892203809763213, 59.41262848674568],
                  [17.892203809763213, 59.414157080424516]]], None, False),
            {
              "landcover": 3,
              "system:index": "17"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.901322141057115, 59.40799004051288],
                  [17.901322141057115, 59.40589328415362],
                  [17.905742421513658, 59.40589328415362],
                  [17.905742421513658, 59.40799004051288]]], None, False),
            {
              "landcover": 3,
              "system:index": "18"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.917372479802232, 59.40185229387586],
                  [17.917372479802232, 59.40058528957054],
                  [17.919303670292955, 59.40058528957054],
                  [17.919303670292955, 59.40185229387586]]], None, False),
            {
              "landcover": 3,
              "system:index": "19"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.025706080799527, 59.39583357416297],
                  [18.025706080799527, 59.39505794802263],
                  [18.027014998798794, 59.39505794802263],
                  [18.027014998798794, 59.39583357416297]]], None, False),
            {
              "landcover": 3,
              "system:index": "20"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.016447095280117, 59.402343764127906],
                  [18.016447095280117, 59.40198879439989],
                  [18.017015723591275, 59.40198879439989],
                  [18.017015723591275, 59.402343764127906]]], None, False),
            {
              "landcover": 3,
              "system:index": "21"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.036756781940884, 59.409322259739454],
                  [18.036756781940884, 59.40821933808673],
                  [18.038033513431973, 59.40821933808673],
                  [18.038033513431973, 59.409322259739454]]], None, False),
            {
              "landcover": 3,
              "system:index": "22"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.04106977403683, 59.40983002842758],
                  [18.04106977403683, 59.408759883383944],
                  [18.043204812412686, 59.408759883383944],
                  [18.043204812412686, 59.40983002842758]]], None, False),
            {
              "landcover": 3,
              "system:index": "23"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.030555514698452, 59.411478859202056],
                  [18.030555514698452, 59.41109668702971],
                  [18.031531838779873, 59.41109668702971],
                  [18.031531838779873, 59.411478859202056]]], None, False),
            {
              "landcover": 3,
              "system:index": "24"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.22510547637924, 59.405251221143196],
                  [18.22510547637924, 59.403394574599446],
                  [18.229954910278167, 59.403394574599446],
                  [18.229954910278167, 59.405251221143196]]], None, False),
            {
              "landcover": 3,
              "system:index": "25"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.229568672180022, 59.406692782403084],
                  [18.229568672180022, 59.40529490571912],
                  [18.23265857696518, 59.40529490571912],
                  [18.23265857696518, 59.406692782403084]]], None, False),
            {
              "landcover": 3,
              "system:index": "26"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.228796195983733, 59.41042745122415],
                  [18.228796195983733, 59.409291806028676],
                  [18.23115653991684, 59.409291806028676],
                  [18.23115653991684, 59.41042745122415]]], None, False),
            {
              "landcover": 3,
              "system:index": "27"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.25158424377426, 59.409269966324835],
                  [18.25158424377426, 59.40800323940825],
                  [18.253215026855315, 59.40800323940825],
                  [18.253215026855315, 59.409269966324835]]], None, False),
            {
              "landcover": 3,
              "system:index": "28"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.282440376281585, 59.406255952135965],
                  [18.282440376281585, 59.40400618705258],
                  [18.28510112762436, 59.40400618705258],
                  [18.28510112762436, 59.406255952135965]]], None, False),
            {
              "landcover": 3,
              "system:index": "29"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.28183956146225, 59.4090515685122],
                  [18.28183956146225, 59.40782851473693],
                  [18.283942413329925, 59.40782851473693],
                  [18.283942413329925, 59.4090515685122]]], None, False),
            {
              "landcover": 3,
              "system:index": "30"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.279221725463714, 59.391138155329855],
                  [18.279221725463714, 59.39022038407913],
                  [18.28085250854477, 59.39022038407913],
                  [18.28085250854477, 59.391138155329855]]], None, False),
            {
              "landcover": 3,
              "system:index": "31"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.858805628165104, 59.35688518815741],
                  [17.858805628165104, 59.35566025129503],
                  [17.864770861014225, 59.35566025129503],
                  [17.864770861014225, 59.35688518815741]]], None, False),
            {
              "landcover": 3,
              "system:index": "32"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.860683517374998, 59.332439573903464],
                  [17.860683517374998, 59.33145454431394],
                  [17.86235721580029, 59.33145454431394],
                  [17.86235721580029, 59.332439573903464]]], None, False),
            {
              "landcover": 3,
              "system:index": "33"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.864545898356443, 59.332133123313604],
                  [17.864545898356443, 59.33088540309],
                  [17.866691665568357, 59.33088540309],
                  [17.866691665568357, 59.332133123313604]]], None, False),
            {
              "landcover": 3,
              "system:index": "34"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.868193702616697, 59.331914228342875],
                  [17.868193702616697, 59.33049137665799],
                  [17.871498184123045, 59.33049137665799],
                  [17.871498184123045, 59.331914228342875]]], None, False),
            {
              "landcover": 3,
              "system:index": "35"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.83562095633984, 59.3441264117299],
                  [17.83562095633984, 59.34169745624415],
                  [17.83914001456738, 59.34169745624415],
                  [17.83914001456738, 59.3441264117299]]], None, False),
            {
              "landcover": 3,
              "system:index": "36"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.84291656486035, 59.34158803974601],
                  [17.84291656486035, 59.33990298116078],
                  [17.845748977580076, 59.33990298116078],
                  [17.845748977580076, 59.34158803974601]]], None, False),
            {
              "landcover": 3,
              "system:index": "37"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.848495559611326, 59.3405157394099],
                  [17.848495559611326, 59.33861177580446],
                  [17.853516654887205, 59.33861177580446],
                  [17.853516654887205, 59.3405157394099]]], None, False),
            {
              "landcover": 3,
              "system:index": "38"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.82815368644238, 59.334059338247684],
                  [17.82815368644238, 59.32933116118502],
                  [17.83394725791455, 59.32933116118502],
                  [17.83394725791455, 59.334059338247684]]], None, False),
            {
              "landcover": 3,
              "system:index": "39"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.84235866538525, 59.32639760951473],
                  [17.84235866538525, 59.323704648867974],
                  [17.846736030497556, 59.323704648867974],
                  [17.846736030497556, 59.32639760951473]]], None, False),
            {
              "landcover": 3,
              "system:index": "40"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.84862430564404, 59.32512778473945],
                  [17.84862430564404, 59.32256609726451],
                  [17.85287292472363, 59.32256609726451],
                  [17.85287292472363, 59.32512778473945]]], None, False),
            {
              "landcover": 3,
              "system:index": "41"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.83038528434277, 59.326726004751904],
                  [17.83038528434277, 59.32309158735917],
                  [17.83639343253613, 59.32309158735917],
                  [17.83639343253613, 59.326726004751904]]], None, False),
            {
              "landcover": 3,
              "system:index": "42"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.8421870040083, 59.31691656562004],
                  [17.8421870040083, 59.314091447685044],
                  [17.849096374430662, 59.314091447685044],
                  [17.849096374430662, 59.31691656562004]]], None, False),
            {
              "landcover": 3,
              "system:index": "43"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.841841254603512, 59.32060021763849],
                  [17.841841254603512, 59.31868418871741],
                  [17.844802413355954, 59.31868418871741],
                  [17.844802413355954, 59.32060021763849]]], None, False),
            {
              "landcover": 3,
              "system:index": "44"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.833236728083737, 59.32049073318122],
                  [17.833236728083737, 59.318673239670744],
                  [17.837785754572995, 59.318673239670744],
                  [17.837785754572995, 59.32049073318122]]], None, False),
            {
              "landcover": 3,
              "system:index": "45"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.82637027300561, 59.32159650999785],
                  [17.82637027300561, 59.32035935136658],
                  [17.82939580477441, 59.32035935136658],
                  [17.82939580477441, 59.32159650999785]]], None, False),
            {
              "landcover": 3,
              "system:index": "46"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.911386801709615, 59.30495284544078],
                  [17.911386801709615, 59.30383009648415],
                  [17.913639857282124, 59.30383009648415],
                  [17.913639857282124, 59.30495284544078]]], None, False),
            {
              "landcover": 3,
              "system:index": "47"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.91147263239809, 59.30640414893083],
                  [17.91147263239809, 59.30563743027495],
                  [17.913296534528218, 59.30563743027495],
                  [17.913296534528218, 59.30640414893083]]], None, False),
            {
              "landcover": 3,
              "system:index": "48"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.164245902422277, 59.40147789136356],
                  [18.164245902422277, 59.39962103802994],
                  [18.168237029436437, 59.39962103802994],
                  [18.168237029436437, 59.40147789136356]]], None, False),
            {
              "landcover": 3,
              "system:index": "49"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.167528926256505, 59.40305067576268],
                  [18.167528926256505, 59.40210046058491],
                  [18.16967469346842, 59.40210046058491],
                  [18.16967469346842, 59.40305067576268]]], None, False),
            {
              "landcover": 3,
              "system:index": "50"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.17594033372721, 59.404710757868614],
                  [18.17594033372721, 59.40360768606332],
                  [18.178064643267003, 59.40360768606332],
                  [18.178064643267003, 59.404710757868614]]], None, False),
            {
              "landcover": 3,
              "system:index": "51"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.16014748704752, 59.398266563181615],
                  [18.16014748704752, 59.39675910009842],
                  [18.162250338915197, 59.39675910009842],
                  [18.162250338915197, 59.398266563181615]]], None, False),
            {
              "landcover": 3,
              "system:index": "52"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.12552726172036, 59.377682086785626],
                  [18.12552726172036, 59.376736625386705],
                  [18.126417755113305, 59.376736625386705],
                  [18.126417755113305, 59.377682086785626]]], None, False),
            {
              "landcover": 3,
              "system:index": "53"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.128681539521875, 59.37983523576857],
                  [18.128681539521875, 59.378632986694015],
                  [18.130215763078393, 59.378632986694015],
                  [18.130215763078393, 59.37983523576857]]], None, False),
            {
              "landcover": 3,
              "system:index": "54"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.12276995085305, 59.382818815234884],
                  [18.12276995085305, 59.381709566346395],
                  [18.124711870179834, 59.381709566346395],
                  [18.124711870179834, 59.382818815234884]]], None, False),
            {
              "landcover": 3,
              "system:index": "55"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.038017095119304, 59.29157749983399],
                  [18.038017095119304, 59.290120084600815],
                  [18.040313066036052, 59.290120084600815],
                  [18.040313066036052, 59.29157749983399]]], None, False),
            {
              "landcover": 3,
              "system:index": "56"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.04288798669035, 59.28893657368063],
                  [18.04288798669035, 59.28844343198375],
                  [18.04366046288664, 59.28844343198375],
                  [18.04366046288664, 59.28893657368063]]], None, False),
            {
              "landcover": 3,
              "system:index": "57"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[18.022524655849285, 59.27946700461082],
                  [18.022524655849285, 59.278403705641374],
                  [18.024713338405437, 59.278403705641374],
                  [18.024713338405437, 59.27946700461082]]], None, False),
            {
              "landcover": 3,
              "system:index": "58"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.858364131567903, 59.20866757305105],
                  [17.858364131567903, 59.20537207616205],
                  [17.866603877661653, 59.20537207616205],
                  [17.866603877661653, 59.20866757305105]]], None, False),
            {
              "landcover": 3,
              "system:index": "59"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.84797861826224, 59.22795083625247],
                  [17.84797861826224, 59.222944384598115],
                  [17.855617549536653, 59.222944384598115],
                  [17.855617549536653, 59.22795083625247]]], None, False),
            {
              "landcover": 3,
              "system:index": "60"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.84720614206595, 59.20335068070559],
                  [17.84720614206595, 59.20247177579459],
                  [17.855016734717317, 59.20247177579459],
                  [17.855016734717317, 59.20335068070559]]], None, False),
            {
              "landcover": 3,
              "system:index": "61"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.823817279456087, 59.22507441214528],
                  [17.823817279456087, 59.222856545556446],
                  [17.826048877356477, 59.222856545556446],
                  [17.826048877356477, 59.22507441214528]]], None, False),
            {
              "landcover": 3,
              "system:index": "62"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.786652591345735, 59.233812720874845],
                  [17.786652591345735, 59.231880825241866],
                  [17.790343310950227, 59.231880825241866],
                  [17.790343310950227, 59.233812720874845]]], None, False),
            {
              "landcover": 3,
              "system:index": "63"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.789356258032747, 59.23073919911932],
                  [17.789356258032747, 59.2291145114572],
                  [17.79330446970267, 59.2291145114572],
                  [17.79330446970267, 59.23073919911932]]], None, False),
            {
              "landcover": 3,
              "system:index": "64"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.788583781836458, 59.22621622783845],
                  [17.788583781836458, 59.22456936609292],
                  [17.79403403055472, 59.22456936609292],
                  [17.79403403055472, 59.22621622783845]]], None, False),
            {
              "landcover": 3,
              "system:index": "65"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.793089892981477, 59.220704418395066],
                  [17.793089892981477, 59.219760074860105],
                  [17.79523566019339, 59.219760074860105],
                  [17.79523566019339, 59.220704418395066]]], None, False),
            {
              "landcover": 3,
              "system:index": "66"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.802445438025423, 59.23949800609466],
                  [17.802445438025423, 59.23765423364748],
                  [17.80523493540091, 59.23765423364748],
                  [17.80523493540091, 59.23949800609466]]], None, False),
            {
              "landcover": 3,
              "system:index": "67"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.78304770242972, 59.25198474509537],
                  [17.78304770242972, 59.24849594100947],
                  [17.78927042734427, 59.24849594100947],
                  [17.78927042734427, 59.25198474509537]]], None, False),
            {
              "landcover": 3,
              "system:index": "68"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.799312617896028, 59.25490277865673],
                  [17.799312617896028, 59.25259908871209],
                  [17.803904559729524, 59.25259908871209],
                  [17.803904559729524, 59.25490277865673]]], None, False),
            {
              "landcover": 3,
              "system:index": "69"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.77081682932181, 59.254946656954],
                  [17.77081682932181, 59.25150203877126],
                  [17.77630999338431, 59.25150203877126],
                  [17.77630999338431, 59.254946656954]]], None, False),
            {
              "landcover": 3,
              "system:index": "70"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.74401592004314, 59.260941517051975],
                  [17.74401592004314, 59.25881372891631],
                  [17.746848332762866, 59.25881372891631],
                  [17.746848332762866, 59.260941517051975]]], None, False),
            {
              "landcover": 3,
              "system:index": "71"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.747406232237964, 59.26032732380907],
                  [17.747406232237964, 59.255413379284974],
                  [17.753586041808276, 59.255413379284974],
                  [17.753586041808276, 59.26032732380907]]], None, False),
            {
              "landcover": 3,
              "system:index": "72"
            }),
        ee.Feature(
            ee.Geometry.Polygon(
                [[[17.754916417479663, 59.25762912949226],
                  [17.754916417479663, 59.25468939007088],
                  [17.75826381433025, 59.25468939007088],
                  [17.75826381433025, 59.25762912949226]]], None, False),
            {
              "landcover": 3,
              "system:index": "73"
            })])

Map.addLayer(water, {}, "water")
Map.addLayer(vegetation, {}, "vegetation")
Map.addLayer(urban, {}, "urban")
Map.addLayer(bare, {}, "bare")

In [11]:
# Combine labelled polygons to one training feature:
trainingFeatures = water.merge(vegetation).merge(urban).merge(bare)
classifierTraining = input.select(inputBands).sampleRegions(
  **{
  'collection': trainingFeatures,
  'properties': ['landcover'],
  'scale': 30,
  'geometries': True
  }
)

# Separate into training & testing sets, but only the testing set is used for Unsupervised Learning Accuracy Assessment:
trainingTesting = classifierTraining.randomColumn()
trainingSet = trainingTesting.filter(ee.Filter.lessThan('random',0.6)) # Not used
testingSet = trainingTesting.filter(ee.Filter.greaterThanOrEquals('random',0.6))

In [12]:
result_classified.bandNames().get(0) # Verify the property inherited from the remap function

In [13]:
# Construct Confusion Matrix:
testVec = result_classified.sampleRegions(
    **{
        'collection': testingSet,
        'scale': 30,
        'geometries': True    
    }
)

confusionMatrix = testVec.errorMatrix('landcover',result_classified.bandNames().get(0))

# Print Confusion Matrix and relevant accuracies:

In [14]:
confusionMatrix

In [15]:
confusionMatrix.accuracy()

In [16]:
confusionMatrix.producersAccuracy()

In [17]:
confusionMatrix.consumersAccuracy()

In [18]:
testingSet.size() # Check size of testing set

# Unsupervised Classification - Final Plot Map:

In [19]:
# Replotting the unsupervised classification result:
Map2 = geemap.Map()
Map2.centerObject(region, 10)
Map2.addLayer(image, vis_params, "Sentinel-2")


Map2.addLayer(
    result_classified, {'min': 0, 'max': 3, 'palette': legend_colors_classified}, 'Unsupervised Classification'
)
Map2.add_legend(
    keys=legend_keys_classified, colors=legend_colors_classified, position='bottomleft'
)
Map2.addLayer(water, {}, "water")
Map2.addLayer(vegetation, {}, "vegetation")
Map2.addLayer(urban, {}, "urban")
Map2.addLayer(bare, {}, "bare")

Map2

Map(center=[59.399925146053874, 18.19999999999986], controls=(WidgetControl(options=['position', 'transparent_…

In [20]:
# Memory cleaning to improve performance:
import gc
gc.collect()

1439